### 6.2.1 Classical networking
This example is simply sending a byte over a socket, using Julia tasks to synchronise execution.

In [ ]:
using Random
using Sockets

The following code is smilar to **Listing 6.1**, providing a "sender" method and a "receiver" method, each using the `Sockets` API

In [ ]:
function runSender(port)
    b = rand(UInt8)
    println("[Sender] Create a connection to port ", port)
    # The sender opens a network socket.
    socket = connect(port)
    println("[Sender] Write a byte: ", b)
    # The sender writes a specific byte to the socket.
    write(socket, b)
    println("[Sender] Wrote a byte: ", b)
    close(socket)
end

In [ ]:
function runReceiver(port)
    # The receiver opens a network socket,
    # listening for incoming requests.
    server = listen(port)
    println("[Receiver] Starting to listen for incoming data at port ", port)
    while true
        b = nothing
        # When a connection is discovered, a direct socket connection
        # is created between the sender and the receiver.
        sock = accept(server)
        while isopen(sock)
            # The receiver reads a byte from the connection.
            b = read(sock, UInt8)
            break
        end
        # The receiver prints the value of the byte
        println("[Receiver] Got a byte: ", b)
        break
    end
end

The next function uses `runSender` and `runReceiver` to exchange data:

In [ ]:
function network_example()
    port = 9755
    # The code for sending bytes is performed in one task.
    sender = @task runSender(port)
    # The code for receiving bytes is performed in another task.
    receiver = @task runReceiver(port)
    # Start the receiver task
    schedule(receiver)
    # Start the sender task
    schedule(sender)
    # Wait until the receiver exits
    wait(receiver)
end

In [ ]:
network_example()